# 3: Additional good outcomes

## Plain English summary
This notebook builds on the scenarios that we considered in a previous notebook. Now that we know the effect of changing scenario on the proportions of patients who meet certain criteria, we can feed that data into a pathway simulator and an outcome model.

The pathway simulator generates a length of time for each patient and for each step in the pathway. The times are selected using the expected distributions of times at each step that we measured previously.

The outcome model takes the final times from onset to treatment for each patient and calculates their outcome. The outcomes are given as mRS (modified Rankin Scale) scores, where 0 is completely independent and 6 is dead. 

We run the pathway and outcome models to generate outcomes for 100 years' worth of patients. By averaging the results over the 100 years, we can find the effect of scenario on the number of additional good outcomes. Here we define additional good outcomes as the extra number compared with the case where nobody receives any treatment for their stroke.

## Notebook setup:

In [1]:
import pandas as pd
import numpy as np
import pickle
from dataclasses import dataclass
from math import sqrt

import matplotlib.pyplot as plt

import stroke_utilities.scenario
from classes.pathway import SSNAP_Pathway

import warnings
warnings.filterwarnings("ignore")

## Set up paths and filenames

In [2]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    output_folder = './stroke_utilities/output/'

paths = Paths()

## Import existing utilities

This hospital performance data for various scenarios was calculated in a previous notebook.

In [3]:
filename = paths.output_folder + 'all_performance_scenarios.csv'
df_performance_scenarios = pd.read_csv(filename, index_col=0)

In [4]:
df_performance_scenarios.T

,1 / lvo / base,1 / nlvo / base,1 / other / base,1 / mixed / base,1 / lvo / benchmark,1 / nlvo / benchmark,1 / other / benchmark,1 / mixed / benchmark,1 / lvo / onset,1 / nlvo / onset,...,119 / other / speed + benchmark,119 / mixed / speed + benchmark,119 / lvo / speed + onset,119 / nlvo / speed + onset,119 / other / speed + onset,119 / mixed / speed + onset,119 / lvo / speed + onset + benchmark,119 / nlvo / speed + onset + benchmark,119 / other / speed + onset + benchmark,119 / mixed / speed + onset + benchmark
stroke_type,lvo,nlvo,other,mixed,lvo,nlvo,other,mixed,lvo,nlvo,...,other,mixed,lvo,nlvo,other,mixed,lvo,nlvo,other,mixed
admissions,153.0,728.0,101.5,982.5,153.0,728.0,101.5,982.5,153.0,728.0,...,69.5,594.5,97.0,428.0,69.5,594.5,97.0,428.0,69.5,594.5
proportion_of_all_with_ivt,0.323529,0.088599,0.0,0.116031,0.375817,0.108585,0.0,0.139949,0.401209,0.119963,...,0.0,0.117297,0.207943,0.07603,0.0,0.089043,0.258661,0.101646,0.0,0.117297
proportion_of_all_with_mt,0.189542,0.011676,0.0,0.038168,0.179739,0.008928,0.0,0.034606,0.242489,0.014049,...,0.0,0.005902,0.030054,0.001184,0.0,0.005902,0.030054,0.001184,0.0,0.005902
proportion_of_mt_with_ivt,0.672414,0.529412,NaN,0.64,0.672414,0.529412,NaN,0.64,0.672414,0.529412,...,NaN,0.555556,0.714286,0.0,NaN,0.555556,0.714286,0.0,NaN,0.555556
proportion1_of_all_with_onset_known_ivt,0.614379,0.484203,0.522167,0.508397,0.614379,0.484203,0.522167,0.508397,0.828872,0.761925,...,0.820144,0.812447,0.865979,0.799065,0.820144,0.812447,0.865979,0.799065,0.820144,0.812447
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.824468,0.547518,0.688679,0.614615,0.824468,0.547518,0.688679,0.614615,0.824468,0.547518,...,0.587719,0.511387,0.690476,0.454678,0.587719,0.511387,0.690476,0.454678,0.587719,0.511387
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.993548,0.943005,0.917808,0.952769,0.993548,0.943005,0.917808,0.952769,0.993548,0.943005,...,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.980519,0.840659,0.880597,0.882051,0.980519,0.840659,0.880597,0.882051,0.980519,0.840659,...,0.888889,0.824295,0.866071,0.793706,0.888889,0.824295,0.866071,0.793706,0.888889,0.824295
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Run models

Find the complete list of stroke teams and scenarios:

In [5]:
stroke_teams = sorted(list(set(df_performance_scenarios['stroke_team'])))

In [6]:
scenario_list = sorted(list(set(df_performance_scenarios['scenario'])))

The following cell looks at each stroke team in turn. It picks out the patients with each stroke type. 

Then separate patient pathways are run for the separate stroke types. The results from the pathways are combined into a single data set.

Some of the pathway results are split off and used for the outcome model. The required results for each patient are their times from onset to treatment, which treatment they received, and which stroke type they have. 

Then the outcome model is run to find out the expected outcome of patients with those features who were treated at those times. The "discrete" outcome model is used, and so the patient must have a pre-stroke mRS score. Rather than sampling this from the original data where there might not be many patients for each stroke type, we create a more representative selection by sampling uniformly from a pre-stroke mRS distribution.

In [7]:
np.random.seed(42)

# How many trials to run:
n_trials = 100

results_df, outcome_results_columns, trial_columns = stroke_utilities.scenario.set_up_results_dataframe()

for stroke_team in stroke_teams:
    for scenario in scenario_list:
        # Get data for one hospital.
        # Squeeze to convert DataFrame to Series.
        lvo_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'lvo') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
        nlvo_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'nlvo') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
        other_data = df_performance_scenarios[(
            (df_performance_scenarios['stroke_team'] == stroke_team) & 
            (df_performance_scenarios['stroke_type'] == 'other') &
            (df_performance_scenarios['scenario'] == scenario)
            )].copy().squeeze()
    
        # Set up trial results dataframe
        trial_df = pd.DataFrame(columns=trial_columns)
        # Set up the pathways with this data...
        pathway_object_dict = stroke_utilities.scenario.set_up_pathway_objects(stroke_team, lvo_data, nlvo_data, other_data)
        for trial in range(n_trials):
            # ... run the pathways...
            combo_trial_dict = stroke_utilities.scenario.run_trial_of_pathways(pathway_object_dict)
            # ... overwrite the results so that nobody has thrombectomy...
            combo_trial_dict['mt_chosen_bool'] = np.array([0] * len(combo_trial_dict['mt_chosen_bool'])) == 1
            # ... and run the clinical outcome model.
            results_by_stroke_type, patient_array_outcomes = (
                stroke_utilities.scenario.run_discrete_outcome_model(combo_trial_dict))

            number_of_patients = len(combo_trial_dict['stroke_type_code'])
            # Patients' mRS if not treated..
            mrs_not_treated = patient_array_outcomes['each_patient_mrs_not_treated']
            # Patients' mRS in this trial...
            mrs_post_stroke = patient_array_outcomes['each_patient_mrs_post_stroke']
            # How many patients were good outcomes? mRS 0 or 1.
            n_good_baseline = len(np.where(mrs_not_treated <= 1)[0])
            n_good_post_stroke = len(np.where(mrs_post_stroke <= 1)[0])
            # Additional good outcomes:
            n_good_additional = n_good_post_stroke - n_good_baseline
            # Convert to outcomes per 1000 patients:
            n_baseline_good_per_1000 = n_good_baseline * (1000.0 / number_of_patients)
            n_additional_good_per_1000 = n_good_additional * (1000.0 / number_of_patients)
    
            result = stroke_utilities.scenario.gather_results_from_trial(
                trial_columns, combo_trial_dict, results_by_stroke_type,
                n_baseline_good_per_1000, n_additional_good_per_1000
                )
            trial_df.loc[trial] = result
    
        summary_trial_results = stroke_utilities.scenario.gather_summary_results_across_all_trials(outcome_results_columns, trial_df)
        summary_trial_results += [f'{stroke_team}']
        summary_trial_results += [scenario.replace(' + ', '_')]
        
        # add scenario results to results dataframe
        results_df.loc[f'{stroke_team} / {scenario}'] = summary_trial_results

## Results

In [8]:
results_df.T

,1 / base,1 / benchmark,1 / onset,1 / onset + benchmark,1 / speed,1 / speed + benchmark,1 / speed + onset,1 / speed + onset + benchmark,2 / base,2 / benchmark,...,118 / speed + onset,118 / speed + onset + benchmark,119 / base,119 / benchmark,119 / onset,119 / onset + benchmark,119 / speed,119 / speed + benchmark,119 / speed + onset,119 / speed + onset + benchmark
Percent_Thrombolysis_(median),10.997963,14.96945,15.835031,22.09776,11.659878,15.885947,17.209776,23.523422,15.743945,16.176471,...,19.02834,14.574899,9.848485,12.962963,9.93266,12.962963,11.279461,14.814815,11.616162,14.646465
Percent_Thrombolysis_(low_5%),9.368635,13.441955,14.460285,19.648676,9.872709,14.144603,15.46334,21.578411,13.49481,14.186851,...,16.578947,12.135628,8.072391,10.43771,8.080808,10.26936,9.427609,12.609428,9.579125,12.786195
Percent_Thrombolysis_(high_95%),12.627291,16.614053,17.718941,24.246436,13.243381,17.627291,19.149695,25.875764,18.16609,18.858131,...,22.074899,16.801619,11.616162,14.814815,11.784512,14.654882,13.468013,17.188552,13.813131,16.835017
Percent_Thrombolysis_(mean),11.02444,14.965377,15.98778,21.994908,11.636456,15.935845,17.298371,23.629328,15.697232,16.243945,...,19.192308,14.534413,9.838384,12.840067,9.907407,12.819865,11.425926,14.900673,11.530303,14.765993
Percent_Thrombolysis_(stdev),0.99435,1.03859,1.120303,1.404006,1.022831,1.164183,1.160683,1.32655,1.494749,1.416449,...,1.752189,1.430233,1.165461,1.345707,1.155855,1.415488,1.224242,1.562406,1.347969,1.342606
Percent_Thrombolysis_(95ci),0.194889,0.20356,0.219575,0.27518,0.200471,0.228176,0.22749,0.259999,0.292965,0.277619,...,0.343423,0.280321,0.228426,0.263754,0.226543,0.277431,0.239947,0.306226,0.264197,0.263146
Baseline_good_outcomes_per_1000_patients_(median),361.507128,362.016293,358.961303,362.525458,361.507128,361.507128,358.961303,361.507128,347.750865,352.941176,...,334.008097,330.97166,358.585859,350.16835,355.218855,355.218855,348.484848,355.218855,353.535354,351.010101
Baseline_good_outcomes_per_1000_patients_(low_5%),337.06721,332.892057,340.98778,341.038697,337.983707,341.14053,338.08554,338.034623,321.799308,326.989619,...,307.692308,299.595142,324.747475,326.515152,324.915825,323.232323,323.148148,326.599327,321.464646,318.097643
Baseline_good_outcomes_per_1000_patients_(high_95%),383.961303,384.01222,384.979633,385.947047,384.979633,386.965377,384.215886,387.016293,377.16263,384.16955,...,366.59919,372.469636,382.323232,378.787879,388.888889,382.154882,375.505051,380.47138,385.690236,377.272727
Baseline_good_outcomes_per_1000_patients_(mean),362.107943,361.323829,360.641548,361.741344,360.509165,363.676171,361.059063,361.456212,349.186851,352.99308,...,334.048583,333.765182,356.043771,351.498316,354.343434,354.444444,349.56229,354.86532,353.417508,351.599327


Save the results to file:

In [9]:
df = results_df

# Round the values to fewer decimal places:
for column in df.columns:
    if column not in ['stroke_team', 'stroke_team_id', 'scenario']:
        df[column] = df[column].astype(float).round(6)
        
df.to_csv(
    f'{paths.output_folder}/scenario_results.csv',
    index=False
    )